# Regression-3: model performance

OLS、リッジ回帰、ランダムフォレスト、勾配ブースティングのアルゴリズム性能を比較してみましょう。データはボストン・ハウジングデータを使いましょう。

In [2]:
# import the data for regression
import pandas as pd
from sklearn.datasets import load_boston
dataset = load_boston()

# set dataframe
X = pd.DataFrame(dataset.data, columns=dataset.feature_names)
y = pd.DataFrame(dataset.target, columns=['y'])

# check the shape
print('----------------------------------------------------------------------------------------')
print('X shape: (%i,%i)' %X.shape)
print('y shape: (%i,%i)' %y.shape)
print('----------------------------------------------------------------------------------------')
print(y.describe())
print('----------------------------------------------------------------------------------------')
print(X.join(y).head())

----------------------------------------------------------------------------------------
X shape: (506,13)
y shape: (506,1)
----------------------------------------------------------------------------------------
                y
count  506.000000
mean    22.532806
std      9.197104
min      5.000000
25%     17.025000
50%     21.200000
75%     25.000000
max     50.000000
----------------------------------------------------------------------------------------
      CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  \
0  0.00632  18.0   2.31   0.0  0.538  6.575  65.2  4.0900  1.0  296.0   
1  0.02731   0.0   7.07   0.0  0.469  6.421  78.9  4.9671  2.0  242.0   
2  0.02729   0.0   7.07   0.0  0.469  7.185  61.1  4.9671  2.0  242.0   
3  0.03237   0.0   2.18   0.0  0.458  6.998  45.8  6.0622  3.0  222.0   
4  0.06905   0.0   2.18   0.0  0.458  7.147  54.2  6.0622  3.0  222.0   

   PTRATIO       B  LSTAT     y  
0     15.3  396.90   4.98  24.0  
1     17.8  396.90   9.14  21

ツリー系のアルゴリズム（ランダムフォレストや勾配ブースティングなど）を除き、通常、多くの機械学習モデルは、入力ベクトルのスケールを統一させる必要があります。ここではその処理をPipelineで組み込んでいます。

In [3]:
# import libraries
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score

# 交差検証(holdout)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# make pipelines for modeling
pipe_ols = Pipeline([('scl',StandardScaler()),('est',LinearRegression())])
pipe_ridge = Pipeline([('scl',StandardScaler()),('est',Ridge())])
pipe_rf = Pipeline([('scl',StandardScaler()),('est',RandomForestRegressor(random_state=42))])
pipe_gbr = Pipeline([('scl',StandardScaler()),('est',GradientBoostingRegressor(random_state=42))])

# build models
pipe_ols.fit(X_train, y_train.as_matrix().ravel())
pipe_ridge.fit(X_train, y_train.as_matrix().ravel())
pipe_rf.fit(X_train, y_train.as_matrix().ravel())
pipe_gbr.fit(X_train, y_train.as_matrix().ravel())

# get R2 score
y_true = y_test.as_matrix().ravel()

# print the performance
print('R2 score of the OLS: %.6f' % r2_score(y_true, pipe_ols.predict(X_test)))
print('R2 score of the Ridge: %.6f' % r2_score(y_true, pipe_ridge.predict(X_test)))
print('R2 score of the RandomForest: %.6f' % r2_score(y_true, pipe_rf.predict(X_test)))
print('R2 score of the GradinetBoostingRegressor: %.6f' % r2_score(y_true, pipe_gbr.predict(X_test)))

R2 score of the OLS: 0.668483
R2 score of the Ridge: 0.668190
R2 score of the RandomForest: 0.886053
R2 score of the GradinetBoostingRegressor: 0.910861


最後にOLSとランダムフォレストの予実プロットを作成してみましょう。大きな誤差が出にくくなっているのが確認できます。

In [4]:
# MUST install plotly before this execution: conda install plotly
import plotly.offline as py
from plotly.graph_objs import *
plotly.offline.init_notebook_mode(connected=False)

trace1 = Scatter(x=pipe_ols.predict(X_test), y=y_true, mode='markers', name='OLS')
trace2 = Scatter(x=pipe_gbr.predict(X_test), y=y_true, mode='markers', name='GradientBoosting')

data = [trace1, trace2]
py.offline.plot(data)

NameError: name 'plotly' is not defined